In [182]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# Create the connection
dt = sqlite3.connect('database.sqlite')

# create the dataframe from a query
df = pd.read_sql_query("SELECT * FROM sqlite_master WHERE type ='table' ", dt)

In [183]:
#Lets look at the features in our dataset
df

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [204]:
top_young_players = pd.read_sql("""
    SELECT Player.player_name, Player.birthday, Player_Attributes.potential, Player_Attributes.overall_rating, Player_Attributes.date,
     Player_Attributes.attacking_work_rate, Player_Attributes.defensive_work_rate,
    strftime('%Y', Player_Attributes.date) - strftime('%Y', Player.birthday) as Age_at_that_time
    FROM Player
    JOIN Player_Attributes ON Player.player_api_id = Player_Attributes.player_api_id
    WHERE strftime('%Y', Player_Attributes.date) - strftime('%Y', Player.birthday) <= 22
    ORDER BY Player_Attributes.potential DESC LIMIT 150
""", dt)
top_young_players['date'] = pd.to_datetime(top_young_players['date'])
top_young_players = top_young_players.sort_values(by='date')

# Reset the index after sorting
top_young_players = top_young_players.reset_index(drop=True)

top_young_players

data_types = data.dtypes

print(data_types)

player_name                    object
birthday                       object
potential                       int64
overall_rating                  int64
date                   datetime64[ns]
attacking_work_rate             int64
defensive_work_rate             int64
Age_at_that_time                int64
dtype: object


In [206]:
import sqlite3
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Encode categorical variables
data = top_young_players.copy()

#print(data_types)
# Encode categorical variables using label encoder with custom mapping
work_rate_mapping = {'low': 1, 'medium': 2, 'high': 3}
label_encoder = LabelEncoder()
label_encoder.fit(['low', 'medium', 'high'])

# Transform the work rate categories using the custom mapping
data['attacking_work_rate'] = data['attacking_work_rate'].map(work_rate_mapping)
data['defensive_work_rate'] = data['defensive_work_rate'].map(work_rate_mapping)
data_types = data.dtypes
print(data)
print(data_types)
# Define features
features = ['attacking_work_rate', 'defensive_work_rate', 'overall_rating']
target_attacking = 'attacking_work_rate'
target_defensive = 'defensive_work_rate'

# Predict Attacking Work Rate
X = top_young_players[features]
y_attacking = top_young_players[target_attacking]

X_train_attacking, X_test_attacking, y_train_attacking, y_test_attacking = train_test_split(X, y_attacking, test_size=0.2, random_state=42)

rf_model_attacking = RandomForestClassifier(random_state=42)
rf_model_attacking.fit(X_train_attacking, y_train_attacking)

rf_y_pred_attacking = rf_model_attacking.predict(X_test_attacking)

rf_accuracy_attacking = accuracy_score(y_test_attacking, rf_y_pred_attacking)
print(f"Accuracy for Attacking Work Rate: {rf_accuracy_attacking}")

# Predict Defensive Work Rate
y_defensive = top_young_players[target_defensive]

X_train_defensive, X_test_defensive, y_train_defensive, y_test_defensive = train_test_split(X, y_defensive, test_size=0.2, random_state=42)

rf_model_defensive = RandomForestClassifier(random_state=42)
rf_model_defensive.fit(X_train_defensive, y_train_defensive)

rf_y_pred_defensive = rf_model_defensive.predict(X_test_defensive)

rf_accuracy_defensive = accuracy_score(y_test_defensive, rf_y_pred_defensive)
print(f"Accuracy for Defensive Work Rate: {rf_accuracy_defensive}")

            player_name             birthday  potential  overall_rating  \
0        Carlos Eduardo  1987-07-18 00:00:00         90              79   
1         Douglas Costa  1990-09-14 00:00:00         90              75   
2                 Diego  1985-02-28 00:00:00         90              83   
3               Lulinha  1990-04-10 00:00:00         91              75   
4         Cesc Fabregas  1987-05-04 00:00:00         90              85   
5            Nuri Sahin  1988-09-05 00:00:00         91              74   
6        Cameron Jerome  1986-10-14 00:00:00         90              72   
7          Arturo Vidal  1987-05-22 00:00:00         90              68   
8          Sergio Ramos  1986-03-30 00:00:00         92              85   
9       Vincent Kompany  1986-04-10 00:00:00         92              86   
10       Yoann Gourcuff  1986-07-11 00:00:00         92              75   
11       Anthony Stokes  1988-07-25 00:00:00         90              54   
12             Anderson  

ValueError: could not convert string to float: 'medium'